In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import display, HTML
from IPython.display import Image
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from pathlib import Path
from typing import Any

from flatland.core.policy import Policy
from flatland.env_generation.env_generator import env_generator
from flatland.trajectories.trajectories import Trajectory
from flatland.evaluators.trajectory_evaluator import TrajectoryEvaluator
from flatland.callbacks.generate_movie_callbacks import GenerateMovieCallbacks

from flatland.utils.seeding import np_random

from flatland.integrations.interactiveai.interactiveai import FlatlandInteractiveAICallbacks

# InteractiveAI integration
Illustrate Flatland callbacks for https://github.com/AI4REALNET/InteractiveAI.
The callbacks create context and events during a scenario run.
If an InteractiveAI instance is up and running, the callbacks  send out HTTP POST requests to InteractiveAI contexts and events REST API endpoints.
In this notebook, we just log the contexts and events that would be sent out.

- The agent positions are sent as context, with geo-coordinates for display on a map.
- Agent malfunctions are sent as events.

## Create Flatland env

In [ ]:
env, observations, _ = env_generator()

## Create position mapping

In [ ]:
# https://opendata.swiss/de/dataset/haltestelle-perronoberflache1
origin_lat = 47.3534027132627
origin_lon = 7.90817796008907
xy_delta = 0.001

In [ ]:
position_to_latlon = {(r,c): (origin_lat - r * xy_delta, origin_lon + c * xy_delta)  for r in range(env.height) for c in range(env.width) }
position_to_latlon

## Create position mapping

In [ ]:
!pip install folium

In [ ]:
import folium

m = folium.Map(location=(origin_lat, origin_lon), zoom_start=14)
    
for rc, coord in position_to_latlon.items():
    folium.Marker(
        location = [coord[0], coord[1]],
        tooltip = f"{rc}",
        icon = folium.Icon(icon="train", prefix="fa", color="blue")
    ).add_to(m)
m

## Trajectory from random policy

In [ ]:
class RandomPolicy(Policy):
    def __init__(self, action_size: int = 5, seed=42):
        super(RandomPolicy, self).__init__()
        self.action_size = action_size
        self.np_random, _ = np_random(seed=seed)

    def act(self, handle: int, observation: Any, **kwargs):
        return self.np_random.choice(self.action_size)

In [ ]:
# requires ffmpeg binary to be installed, see https://github.com/kkroening/ffmpeg-python
!pip install folium python-ffmpeg

In [ ]:
# requires ffmpeg binary to be installed, see https://github.com/kkroening/ffmpeg-python
trajectory = Trajectory.create_from_policy(policy=RandomPolicy(), data_dir=Path("./data").resolve(), callbacks=GenerateMovieCallbacks())
trajectory

In [ ]:
for i in range(env._max_episode_steps):
    display(Image(Path(f"./data/outputs/flatland_frame_{i:04d}.png").resolve()))

## Run trajectory with InteractiveAI callbacks

In [ ]:
# set collect_only=False to send out HTTP POST requests to InteractiveAI contexts and events REST API endpoints
cb = FlatlandInteractiveAICallbacks(position_to_latlon, collect_only=True, step_to_millis=0.02)
cb

In [ ]:
TrajectoryEvaluator(trajectory, cb).evaluate()

In [ ]:
cb.contexts

In [ ]:
cb.events